# **1. Setup and Imports**

In [2]:
!pip install transformers datasets torch sklearn
!pip install datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import Dataset
from google.colab import drive

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/trainset/power/power-tr-train.tsv'
data = pd.read_csv(file_path, delimiter='\t', encoding='utf-8')
data = data.dropna(subset=['text', 'text_en', 'label'])




  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Printing the Data Set**

In [3]:
print(data.head())

        id                           speaker sex  \
0  tr18146  ca2031caa4032c51980160359953d507   M   
1  tr18147  4cee0addb3c69f6866869b180f90d45f   M   
2  tr18148  b3d7f76d74ec268492f8190ca123a6b2   M   
3  tr18149  722efac7138c8197a9d1e97eed3a8b18   M   
4  tr18150  fcc61122f3553c57ae207adeb1a1af84   M   

                                                text  \
0  Yeni yasama döneminin ülkemiz için, milletimiz...   
1  Sayın Başkan, değerli milletvekilleri; bugün, ...   
2  Sayın Başkanım, öncelikle yüce Meclisin Başkan...   
3  24’üncü Dönem Meclis Başkanlığına seçilmenizde...   
4  Usul tartışmasında 2 kişi lehte 2 kişi aleyhte...   

                                             text_en  label  
0  Mr. President, dear lawmakers, I salute you, a...      0  
1  Mr. President, members of lawmakers, as I spea...      0  
2  Mr. President, I'm here to share with you the ...      0  
3  Mr. President, under the principles determined...      0  
4  Two in favour of two in the legal deb

# **3. Tokenization and Dataset Preparation**

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_and_prepare(data, text_column):
    texts = data[text_column].tolist()
    encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    return {
        'input_ids': encodings['input_ids'].detach().numpy(),
        'attention_mask': encodings['attention_mask'].detach().numpy(),
        'labels': data['label'].to_numpy()
    }


encoded_data_en = tokenize_and_prepare(data, 'text_en')
encoded_data_orig = tokenize_and_prepare(data, 'text')
train_data, test_data = train_test_split(data, test_size=0.1, stratify=data['label'], random_state=42)

def create_dataset(encoded_data, indices):
    return Dataset.from_dict({
        'input_ids': np.array([encoded_data['input_ids'][i] for i in indices]),
        'attention_mask': np.array([encoded_data['attention_mask'][i] for i in indices]),
        'labels': np.array([encoded_data['labels'][i] for i in indices])
    })

train_dataset_en = create_dataset(encoded_data_en, train_data.index)
test_dataset_en = create_dataset(encoded_data_en, test_data.index)
train_dataset_orig = create_dataset(encoded_data_orig, train_data.index)
test_dataset_orig = create_dataset(encoded_data_orig, test_data.index)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

# **4. Model Training and Evaluation Function**

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


training_args = TrainingArguments(
    output_dir='./results_task2',
    num_train_epochs=3,
    evaluation_strategy='epoch',
    save_strategy="epoch",
    per_device_train_batch_size=8,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    logging_dir='./logs_task2',
    report_to='none'  # Disable external logging
)

# Train and evaluate the model for English text
model_en = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
trainer_en = Trainer(
    model=model_en,
    args=training_args,
    train_dataset=train_dataset_en,
    eval_dataset=test_dataset_en,
    compute_metrics=compute_metrics
)
trainer_en.train()
eval_results_en = trainer_en.evaluate()
print("Evaluation results for English model:", eval_results_en)

# Train and evaluate the model for Original Language text
model_orig = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
trainer_orig = Trainer(
    model=model_orig,
    args=training_args,
    train_dataset=train_dataset_orig,
    eval_dataset=test_dataset_orig,
    compute_metrics=compute_metrics
)
trainer_orig.train()
eval_results_orig = trainer_orig.evaluate()
print("Evaluation results for Original Language model:", eval_results_orig)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.702300,0.698293,0.485911,0.000000,0.000000,0.000000
2,0.698700,0.693227,0.485911,0.000000,0.000000,0.000000
3,0.693700,0.692752,0.514089,0.679073,0.514089,1.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation results for English model: {'eval_loss': 0.692752480506897, 'eval_accuracy': 0.5140885566417481, 'eval_f1': 0.6790733004177744, 'eval_precision': 0.5140885566417481, 'eval_recall': 1.0, 'eval_runtime': 52.0286, 'eval_samples_per_second': 33.424, 'eval_steps_per_second': 4.19, 'epoch': 3.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.698500,0.699093,0.485911,0.000000,0.000000,0.000000
2,0.696100,0.692826,0.514089,0.679073,0.514089,1.000000
3,0.693000,0.679850,0.638298,0.585912,0.712000,0.497763


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation results for Original Language model: {'eval_loss': 0.6798498034477234, 'eval_accuracy': 0.6382978723404256, 'eval_f1': 0.5859117840684661, 'eval_precision': 0.712, 'eval_recall': 0.49776286353467564, 'eval_runtime': 52.1595, 'eval_samples_per_second': 33.34, 'eval_steps_per_second': 4.179, 'epoch': 3.0}


# **5. Zero-Shot Inference and Evaluation Function**

In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sample_texts_en = test_data['text_en'].dropna().sample(5).tolist()
sample_texts_orig = test_data['text'].dropna().sample(5).tolist()

predictions_en = zero_shot_classifier(sample_texts_en, candidate_labels=["coalition", "opposition"])
predictions_orig = zero_shot_classifier(sample_texts_orig, candidate_labels=["coalition", "opposition"])
print("Zero-shot classification results for English text:", predictions_en)
print("Zero-shot classification results for original language text:", predictions_orig)
def evaluate_zero_shot_batch(model, texts, true_labels, candidate_labels):

    texts = texts.dropna().tolist()
    true_labels = true_labels[:len(texts)]

    batch_predictions = model(texts, candidate_labels=candidate_labels, batch_size=16)
    preds = [0 if pred['labels'][0] == "coalition" else 1 for pred in batch_predictions]


    acc = accuracy_score(true_labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average='binary')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

zero_shot_results_en = evaluate_zero_shot_batch(
    zero_shot_classifier, test_data['text_en'], test_data['label'], ["coalition", "opposition"]
)

zero_shot_results_orig = evaluate_zero_shot_batch(
    zero_shot_classifier, test_data['text'], test_data['label'], ["coalition", "opposition"]
)


print("Zero-shot evaluation results for English text:", zero_shot_results_en)
print("Zero-shot evaluation results for Original Language text:", zero_shot_results_orig)


Device set to use cuda:0


Zero-shot classification results for English text: [{'sequence': "Thank you, Mr. President. <p> Dear lawmakers, when we asked for the number of decisions, a friend in mixed humor told me why you're doing this, what's the point? Therefore, seeking the sufficient number of decisions is once an instrument in the hands of the opposition. He'll have to use it to negotiate from time to time, one. <p> Secondly, our MPs, such as Layla Zana, have been demoted here because of your absentee vote. <p> Now, you're both going to reduce the position of deputy Leyla Zana and the rest of our friends for lack of time, and you won't even bother to come in this building, even though you're on this campus. I don't have anything to say about your deployment by your headquarters or groups outside Ankara, which we do, but the MPs on campus will be kind enough to come to this General Assembly. <p> Now, as for the <PARTY> group proposal, I'll tell you in advance, we'll support it. Of course, the number of missi